# Paperspace用 ComfyUI 起動ノートブック（最終安全版）

**特徴：**
- 必要なツールやライブラリを全て自動でインストールします。
- 必須拡張機能「ComfyUI-Manager」も自動でインストールされます。
- 基本モデルも自動でダウンロードされるので、すぐに画像生成を試せます。
- あなたのGoogleドライブから、好きなモデルを追加することもできます。

**使い方：**
1. 上のメニューから `実行` → `すべてのセルを実行` をクリックしてください。
2. 初回は15分以上かかります。じっくりお待ちください。
3. 最後のセルでComfyUIが起動します。

**アクセス方法：**
最後のセルが実行されたら、Paperspace画面の **ポート(8123)へのリンク** をクリックしてください。

In [ ]:
# Step-1： 環境の準備（ビルドツール等のインストール）
!apt-get update -y
!apt-get install -y git wget build-essential cmake libgl1 libglib2.0-0

In [ ]:
# Step-2： ComfyUI本体と拡張機能（Manager）のインストール
import os
COMFYUI_DIR = '/notebooks/ComfyUI'
if not os.path.exists(COMFYUI_DIR):
    !git clone --depth 1 https://github.com/comfyanonymous/ComfyUI.git {COMFYUI_DIR}
else:
    !cd {COMFYUI_DIR} && git pull

MANAGER_DIR = os.path.join(COMFYUI_DIR, 'custom_nodes', 'ComfyUI-Manager')
if not os.path.exists(MANAGER_DIR):
    !git clone https://github.com/ltdrdata/ComfyUI-Manager.git {MANAGER_DIR}

In [ ]:
# Step-3： PyTorchと依存ライブラリのインストール（最新版・エラー対策済み）
import subprocess
cuda_available = subprocess.run(['nvidia-smi', '-L'], capture_output=True).stdout.decode('utf-8')

print("インストール前の環境をクリーンアップします...")
!pip uninstall torch torchvision torchaudio xformers -y
!pip cache purge

print("ComfyUIが必要とする、最新のPyTorchをインストールします...")
if cuda_available:
    !pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu118
else:
    !pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cpu

print("その他のライブラリをインストールしています...")
!pip install --upgrade pip wheel setuptools gdown
!pip install -r {COMFYUI_DIR}/requirements.txt --prefer-binary

# Numpy, Pillow, toml の問題をここで最終解決します
!pip install --upgrade numpy Pillow toml

if cuda_available:
    !pip install xformers==0.0.25.post1

In [ ]:
# Step-4： 基本モデルの自動ダウンロード
TARGET_DIR=f"{COMFYUI_DIR}/models/checkpoints"
os.makedirs(TARGET_DIR, exist_ok=True)
MODEL_URL="https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors"
!wget -nc -O {TARGET_DIR}/sd15_v1.5.safetensors {MODEL_URL}

### 【オプション】Step-5： あなたのモデルをGoogleドライブから追加
自分の好きなモデルを使いたい場合は、このセルを編集して実行してください。

In [ ]:
# ▼▼▼ ここを編集してください ▼▼▼
gdrive_url = "ここにGoogleドライブの共有リンクを貼り付けてください"
output_filename = "my_model.safetensors"
# ▲▲▲ ここまで ▲▲▲

if "drive.google.com" in gdrive_url:
    !gdown {gdrive_url} -O {TARGET_DIR}/{output_filename}
else:
    print("⚠️ 正しいGoogleドライブの共有リンクを貼り付けてください。")

In [ ]:
# Step-6： ComfyUIの起動（安全な方法）
# Paperspaceの規約に違反しない、最も安全な方法で起動します。
!cd {COMFYUI_DIR} && python main.py --listen --port 8123